In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### read from wos data then transform to node and edge csv

In [2]:
import os
import pandas as pd
import numpy as np
import itertools

In [14]:
path = os.getcwd()
scient_coauthor_path = path+'\\data\\scientometrics\\co-author\\'
scient_cooccurr_path = path+'\\data\\scientometrics\\co-occurrence\\'
jasist_coauthor_path = path+'\\data\\jasist\\co-author\\'
jasist_cooccurr_path = path+'\\data\\jasist\\co-occurrence\\'

output_scient_coauthor_path = path+"\\data transformation output\\scientometrics\\co-author\\"
output_scient_cooccurr_path = path+"\\data transformation output\\scientometrics\\co-occurrence\\"
output_jasist_coauthor_path = path+"\\data transformation output\\jasist\\co-author\\"
output_jasist_cooccurr_path = path+"\\data transformation output\\jasist\\co-occurrence\\"

f_paths = [#scient_coauthor_path\
           scient_cooccurr_path\
           #, jasist_coauthor_path\
           , jasist_cooccurr_path\
          ]
output_f_paths = [#output_scient_coauthor_path\
                  output_scient_cooccurr_path\
                  #, output_jasist_coauthor_path\
                  , output_jasist_cooccurr_path\
                 ]

In [1]:
def list_wos_files_paths(files_path):
    files = os.listdir(files_path)
    wos_txt = []
    for file in files:
        fs = os.listdir(files_path+file)
        for f in fs:
#             if f.endswith(').txt'):
                wos_txt.append(files_path+file+"\\"+f)
    return files, wos_txt

def read_wosfile_to_df(times, wos_filepaths):
    df_all_time = []
    for t in times:
        df = pd.DataFrame()
        for i, file in enumerate(wos_filepaths):
            if t in file:
                # print(i, t)
                df_iter = pd.read_csv(file, sep='\t', index_col=False)
                df = df.append(df_iter)
        df_all_time.append(df)
    return df_all_time

## save wos txt files to csv in data transformation output

In [16]:
# def main(f_path):
#     times, wos_filepaths = list_wos_files_paths(f_path)
#     df_all_times = read_wosfile_to_df(times, wos_filepaths)
    
#     file_pref = '_'.join(f_path.split('\\')[-3:-1])
#     for i, df in enumerate(df_all_times):
#         df.to_csv(output_path+file_pref+"_{}.csv".format(times[i]))
#         print("file saved:\t {}".format(file_pref+"_{}.csv".format(times[i])))
    
# for f_path in f_paths:
#     print("-----{}------".format(f_path))
#     main(f_path)

In [17]:
# for f_i, f_path in enumerate(f_paths):
#     print('\n', "-"*10, f_path)
#     times, wos_filepaths = list_wos_files_paths(f_path)
#     df_all_times = read_wosfile_to_df(times, wos_filepaths)
        
#     for i, df in enumerate(df_all_times):
#         df.to_csv(output_f_paths[f_i]+"{}_wos.csv".format(times[i]), index=None)
#         print("file saved:\t {}".format(output_f_paths[f_i]+"{}_wos.csv".format(times[i])))

## generate keywords table with id

In [10]:
def generate_kw(df_all_time):
    afs = list(df_all_time['DE'])
    new_afs = []
    for af in afs:
        if type(af)!=float:
            new_afs.append(af.split('; '))
        else:
            new_afs.append([])
    # flatten list
    all_afs = [name for names in new_afs for name in names]
    
    return new_afs, sorted(list(set(all_afs)))

## generate author full name table with id

In [19]:
def generate_af(df_all_time):
    aus = list(df_all_time['AU'])
    afs = list(df_all_time['AF'])

    new_aus, new_afs = [], []

    for au in aus:
        new_aus.append(au.split('; '))
    for af in afs:
        new_afs.append(af.split('; '))
    
    # flatten list
    all_aus = [name for names in new_aus for name in names]
    all_afs = [name for names in new_afs for name in names]

    print("List of origin all names:\t {} AU; \t {} AF".format(len(all_aus), len(all_afs)))
    print("Set  of origin all names:\t {} AU; \t {} AF".format(len(list(set(all_aus))), len(list(set(all_afs))))) # full name has more values than abr. name
    
    return new_afs, sorted(list(set(all_afs)))

In [20]:
# combine all authors in all times and set it
def generate_all_names(all_afs_set):
    all_names = []
    for afs_set in all_afs_set:
        all_names+=afs_set
    all_names = sorted(list(set(all_names)))
    
    # all_nodes_table
    df_all_names = pd.DataFrame(columns=['id', 'label'])
    df_all_names['label'] = all_names
    df_all_names['id'] = list(range(1, len(all_names)+1))

    return all_names, df_all_names

In [13]:
def generate_all_id_cooccurs(all_afs, all_names):
    all_id_cooccurs = []

    for afs in all_afs:
        id_cooccurs_list = []
        for cooccurs in afs:
            id_cooccurs = []
            for name in cooccurs:
                id_cooccurs.append(all_names.index(name)+1)
            id_cooccurs_list.append(id_cooccurs)
        all_id_cooccurs.append(id_cooccurs_list)
        
    return all_id_cooccurs

def generate_all_pair_lists(all_id_cooccurs):
    all_pair_lists = []
    for cooccurence_list in all_id_cooccurs:
        all_pair_list = []
        for cooccur in cooccurence_list:
            pair_list = []
            for pair in itertools.combinations(cooccur, 2):
                pair_list.append(pair)

            all_pair_list+=pair_list

        all_pair_lists.append(all_pair_list)
    return all_pair_lists
        
        
def generate_df_edges(all_pair_lists):    
    df_edges = []
    for all_pair_list in all_pair_lists:
        print("pair length:\t", len(all_pair_list))

        df_edge = pd.DataFrame(columns=['source', 'target', 'weight'])
        source_list, target_list = [], []

        for pair in all_pair_list:
            source_list.append(pair[0])
            target_list.append(pair[1])

        df_edge['source'] = source_list
        df_edge['target'] = target_list
        df_edge['weight'] = [1]*len(all_pair_list)
        df_edge = df_edge.sort_values(by=['source', 'target'])
        df_edge.index = list(range(len(df_edge)))

        df_edges.append(df_edge)

    return df_edges

### print drop duplicates should be the same length as after generate_weight function

In [15]:
def generate_weight(df):
    # x = pd.DataFrame(columns=['source', 'target', 'weight'])
    # x['source'] = [1,1,1,2,3,3,4,5,5,5]
    # x['target'] = [2,2,2,3,4,4,5,1,1,1]
    # x['weight'] = [1,1,1,1,1,1,1,1,1,1]
    # x = df_edges[0][['source', 'target', 'weight']]
    
    x = df[['source', 'target', 'weight']]
    x.loc[len(x.index)] = ['end', 'end', 1] # use end index to subtract the weight by index number in the end

    y = x.drop_duplicates(subset=['source', 'target'])
    y_index = list(y.index)
    # print(y_index)
    new_weight = []
    for i, index in enumerate(y_index):
        if i+1<len(y_index):
            new_weight.append(y_index[i+1]-y_index[i])
    y = y[y['source']!='end']
    y['weight'] = new_weight
    y = y.sort_values(by=['source', 'target'])
    
    return y

## Main

## main for author

In [24]:
def main_au(f_path):
    times, wos_filepaths = list_wos_files_paths(f_path)
    df_all_times = read_wosfile_to_df(times, wos_filepaths)

    all_afs, all_afs_set = [], []
    for df_all_time in df_all_times:
        afs, afs_set = generate_af(df_all_time)
        all_afs.append(afs)
        all_afs_set.append(afs_set)
        
    all_names = [name for names in all_afs_set for name in names]
    all_names = list(set(all_names))
    print("\n Nodes of all time: \t", len(all_names))

    df_nodes = []
    for nodes in all_afs_set:
        df_node = pd.DataFrame(columns=['id', 'label'])

        node_id_list, node_label_list = [], []
        for name in nodes:
            node_id_list.append(all_names.index(name)+1)
            node_label_list.append(name)

        df_node['id'] = node_id_list
        df_node['label'] = node_label_list
        df_nodes.append(df_node)

    all_id_cooccurs = generate_all_id_cooccurs(all_afs, all_names)
    all_pair_lists = generate_all_pair_lists(all_id_cooccurs)
    print()
    df_edges = generate_df_edges(all_pair_lists)
    
    print("\nUnique edge before generate weight: \n ")
    for i, df in enumerate(df_edges):
        print(times[i], "\t", len(df.drop_duplicates(subset=['source', 'target'])), 'edges')

    print("\nUnique edge after generate weight: \n ")
    df_weight_edges = []
    for i, df_edge in enumerate(df_edges):
        df_iter = generate_weight(df_edge)
        df_weight_edges.append(df_iter)
        print(times[i], "\t", len(df_iter), 'edges')
        
    return times, df_all_times, df_nodes, df_weight_edges

## main for keywords

In [7]:
def main_kw(f_path):
    times, wos_filepaths = list_wos_files_paths(f_path)
    df_all_times = read_wosfile_to_df(times, wos_filepaths)

    all_afs, all_afs_set = [], []
    for df_all_time in df_all_times:
        afs, afs_set = generate_kw(df_all_time)
        all_afs.append(afs)
        all_afs_set.append(afs_set)
        
    all_names = [name for names in all_afs_set for name in names]
    all_names = list(set(all_names))
    print("\n Nodes of all time: \t", len(all_names))

    df_nodes = []
    for nodes in all_afs_set:
        df_node = pd.DataFrame(columns=['id', 'label'])

        node_id_list, node_label_list = [], []
        for name in nodes:
            node_id_list.append(all_names.index(name)+1)
            node_label_list.append(name)

        df_node['id'] = node_id_list
        df_node['label'] = node_label_list
        df_nodes.append(df_node)

    all_id_cooccurs = generate_all_id_cooccurs(all_afs, all_names)
    all_pair_lists = generate_all_pair_lists(all_id_cooccurs)
    print()
    df_edges = generate_df_edges(all_pair_lists)
    
    print("\nUnique edge before generate weight: \n ")
    for i, df in enumerate(df_edges):
        print(times[i], "\t", len(df.drop_duplicates(subset=['source', 'target'])), 'edges')

    print("\nUnique edge after generate weight: \n ")
    df_weight_edges = []
    for i, df_edge in enumerate(df_edges):
        df_iter = generate_weight(df_edge)
        df_weight_edges.append(df_iter)
        print(times[i], "\t", len(df_iter), 'edges')
        
    return times, df_all_times, df_nodes, df_weight_edges

### save

In [11]:
path = os.getcwd()
f_paths = [path+"\\data\\scientometrics_2010-2019\\"]
output_f_paths = [path+"\\data transformation output\\scientometrics\\co-author\\"]

In [16]:
times, df_all_times, df_nodes, df_weight_edges = main_kw(f_paths[0])


 Nodes of all time: 	 7512

pair length:	 18083
pair length:	 3887
pair length:	 8135
pair length:	 12634
pair length:	 16566

Unique edge before generate weight: 
 
2010-2015 	 17029 edges
2016-2016 	 3811 edges
2016-2017 	 7921 edges
2016-2018 	 12232 edges
2016-2019 	 15957 edges

Unique edge after generate weight: 
 
2010-2015 	 17029 edges
2016-2016 	 3811 edges
2016-2017 	 7921 edges
2016-2018 	 12232 edges
2016-2019 	 15957 edges


In [27]:
arr = df_all_times[0]['DE']

In [31]:
df_all_time = df_all_times[0]

In [32]:
y1, y2 = generate_kw(df_all_time)

In [46]:
arr.iloc[9]

'Women; Networks; Advice; Support; Field of science'

In [76]:
for string in arr:
    if type(string)!=float:
        
        if "'Journal impact factor" in string:
            print(string)

'Journal impact factor; Impact factor; Journal citation reports; Citation analysis; Scientific journals; Citation indicators; Bibliometrics; Influence measures; Criticism; Research evaluation


In [69]:
s = int(input())
e = s+20
for i in range(s, e):
    print(i, '\t', y1[i])

260
260 	 ['Women scientists', 'Russia', 'Bibliometrics', 'Surnames', 'Citations']
261 	 ['Composite indicators', 'Rankings', 'Benchmarking', 'Higher Education Institutions', 'Weighting schemes', 'Simulation techniques']
262 	 ['Durability', 'Obsolescence', 'Bibliometric indicators', 'Individual level analysis', 'Micro-level analysis', 'Mendel syndrome']
263 	 ['Research productivity', 'Academic psychologists', 'Measurement']
264 	 ['Publication analysis', 'First quartile journals', 'Overall prestige', 'Ranking methods', 'Axiomatic index', 'Longitudinal analysis']
265 	 ['Citation analysis', 'Matthew effect', 'Scholarly communication']
266 	 ['Patent citation', 'Electronic-paper display technology', 'Emerging field']
267 	 ['Bibliometrics', 'Biomedical journals', 'Impact factor', 'Nobel prize', 'Publication productivity', 'Surnames']
268 	 ['Astronomical journals', 'Publications', 'Citations']
269 	 ['Citations', 'Co-authors', 'Pareto distribution']
270 	 ['Bibliometric indicator', 'Ci

In [ ]:
def generate_kw(df_all_time):
    afs = list(df_all_time['DE'])
    new_afs = []
    for af in afs:
        if type(af)!=float:
            new_afs.append(af.split('; '))
        else:
            new_afs.append([])
    # flatten list
    all_afs = [name for names in new_afs for name in names]
    
    return new_afs, sorted(list(set(all_afs)))

In [31]:
for f_i, f_path in enumerate(f_paths):
    print('\n', "-"*10, f_path)
    times, df_all_times, df_nodes, df_weight_edges = main_au(f_path)
    
    for i, df in enumerate(df_all_times):
        df.to_csv(output_f_paths[f_i]+"{}_wos.csv".format(times[i]), index=None)
    
    for i, df_node in enumerate(df_nodes):
        df_node.to_csv(output_f_paths[f_i]+"{}_node.csv".format(times[i]), index=None)
    
    for i, df_edge in enumerate(df_weight_edges):
        df_edge.to_csv(output_f_paths[f_i]+"{}_edge.csv".format(times[i]), index=None)


 ---------- C:\Users\Liser\Desktop\linchengwei_link_prediction\data\scientometrics_2010-2019\
List of origin all names:	 4375 AU; 	 4375 AF
Set  of origin all names:	 2744 AU; 	 2900 AF
List of origin all names:	 980 AU; 	 980 AF
Set  of origin all names:	 861 AU; 	 863 AF
List of origin all names:	 2056 AU; 	 2056 AF
Set  of origin all names:	 1644 AU; 	 1668 AF
List of origin all names:	 3036 AU; 	 3036 AF
Set  of origin all names:	 2258 AU; 	 2305 AF
List of origin all names:	 3916 AU; 	 3916 AF
Set  of origin all names:	 2814 AU; 	 2876 AF

 Nodes of all time: 	 5190

pair length:	 5478
pair length:	 1289
pair length:	 2733
pair length:	 4014
pair length:	 5258

Unique edge before generate weight: 
 
2010-2015 	 4865 edges
2016-2016 	 1248 edges
2016-2017 	 2584 edges
2016-2018 	 3721 edges
2016-2019 	 4813 edges

Unique edge after generate weight: 
 
2010-2015 	 4865 edges
2016-2016 	 1248 edges
2016-2017 	 2584 edges
2016-2018 	 3721 edges
2016-2019 	 4813 edges


In [145]:
for f_i, f_path in enumerate(f_paths):
    print('\n', "-"*10, f_path)
    times, df_all_times, df_nodes, df_weight_edges = main(f_path)
    
    for i, df in enumerate(df_all_times):
        df.to_csv(output_f_paths[f_i]+"{}_wos.csv".format(times[i]), index=None)
    
    for i, df_node in enumerate(df_nodes):
        df_node.to_csv(output_f_paths[f_i]+"{}_node.csv".format(times[i]), index=None)
    
    for i, df_edge in enumerate(df_weight_edges):
        df_edge.to_csv(output_f_paths[f_i]+"{}_edge.csv".format(times[i]), index=None)
    
    


 ---------- C:\Users\Liser\Desktop\linchengwei_link_prediction\data\scientometrics\co-occurrence\

 Nodes of all time: 	 7692

pair length:	 18460
pair length:	 22504
pair length:	 26791
pair length:	 31457
pair length:	 35745

Unique edge before generate weight: 
 
2010-2015 	 17305 edges
2010-2016 	 21010 edges
2010-2017 	 25032 edges
2010-2018 	 29350 edges
2010-2019 	 33293 edges

Unique edge after generate weight: 
 
2010-2015 	 17305 edges
2010-2016 	 21010 edges
2010-2017 	 25032 edges
2010-2018 	 29350 edges
2010-2019 	 33293 edges

 ---------- C:\Users\Liser\Desktop\linchengwei_link_prediction\data\jasist\co-occurrence\

 Nodes of all time: 	 386

pair length:	 714
pair length:	 1124
pair length:	 1124
pair length:	 1124
pair length:	 1124

Unique edge before generate weight: 
 
2010-2015 	 650 edges
2010-2016 	 1018 edges
2010-2017 	 1018 edges
2010-2018 	 1018 edges
2010-2019 	 1018 edges

Unique edge after generate weight: 
 
2010-2015 	 650 edges
2010-2016 	 1018 edges
20